# CM50270 Reinforcement Learning
## Coursework Part 1: Value Iteration

In this exercise, you will implement the value iteration algorithm for three closely related, but different, gridworld environments.

**Total number of marks:** 20 marks.

**What to submit:** Your completed Jupyter notebook (.ipynb file) which should include **all** of your source code. Please **do not change the file name or compress/zip your submission**. Please do not include any identifying information on the files you submit. This coursework will be marked **anonymously**.

**Where to submit:** CM50270 Moodle Page.

You are required to **work individually**. You are welcome to discuss ideas with others but you must design your own implementation and **write your own code**.

**Do not plagiarise**. Plagiarism is a serious academic offence. For details on what plagiarism is and how to avoid it, please visit the following webpage: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

If you are asked to use specific variable names, data-types, function signatures and notebook cells, **please ensure that you follow these instructions**. Not doing so will cause the automarker to reject your work, and will assign you a score of zero for that question. **If the automarker rejects your work because you have not followed the instructions, you may not get any credit for your work**.

Please **do not use any non-standard, third-party libraries** apart from numpy and matplotlib. **If we are unable to run your code because you have used unsupported external libraries, you may not get any credit for your work.**

Please remember to **save your work regularly**.

Please be sure to **restart the kernel and run your code from start-to-finish** (Kernel → Restart & Run All) before submitting your notebook. Otherwise, you may not be aware that you are using variables in memory that you have deleted.

Your total runtime must be less than **1 minute** on the University's computers. Otherwise, you may not get credit for your work. You can run your code on the university's computers remotely using [UniDesk](https://bath.topdesk.net/tas/public/ssp/content/detail/knowledgeitem?unid=ff3266344c1d4eb2acb227cc9e3e1eee).

## Introduction
In this coursework, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but closely related) Markov Decision Processes. For your reference, the pseudo-code for the Value Iteration algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 2018, pp. 83).

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The symbol $r$ represents the immediate reward on transition from state $s$ to the next state $s'$ via action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this kind of environment from the lectures. The grid squares in the figure are numbered as shown. In all exercises, the following are true: 

**Actions available:** The agent has four possible actions in each grid square. These are *west*, *north*, *south*, and *east*. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold (from a neighbouring grid square), the agent collects the gold. Note that, in order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square.

**Hitting the bomb:** On arrival at a grid square that contains a bomb (from a neighbouring grid square), the agent activates the bomb. 

**Terminal states:** The game terminates when all gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to $1 \times 10 ^{-10}$. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold (from a neighbouring grid square) is $-1 + 10 = +9$. 

## Exercise 1: Deterministic Environment (0 Marks)

In this exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment.

Compute the optimal policy using Value Iteration. 

Your need to produce two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index $i$ representing grid cell $i$ (see figure above). Both arrays should be accessible in the "solution cell" below!

The array `policy` should be a numpy array of strings that specifies an optimal action at each grid location. Please use the abbreviations `"n"`, `"e"`, `"s"`, and `"w"` for the four actions. As an example, the value of `policy` at index `0` needs to give `"n"`, if _north_ is an optimal action in cell 0. The policy for a terminal state can be any action. If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value under the optimal policy).

In [1]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.
import numpy as np

class Deterministic:
    def __init__(self):
        # gridworld dimensions
        self.num_rows = 5
        self.num_cols = 5
        self.num_cells = self.num_cols * self.num_rows

        # item positions
        self.bomb_positions = np.array([18])
        self.gold_positions = np.array([23])
        self.terminal_states = np.array([self.bomb_positions, self.gold_positions])
        
        # reward is -1 for moving into grid square with an additional reward of +/- 10 for the terminal states
        self.rewards = np.array([-1]*self.num_cells)
        self.rewards[self.bomb_positions] += -10
        self.rewards[self.gold_positions] += 10

        self.available_actions = ["n", "e", "s", "w"]

        # state values are initially zero and policy has been initialised with optimal policy "n"
        self.v = np.zeros(self.num_cells)
        self.policy = np.array(["n"]*self.num_cells)
        self.theta = 1e-10
        self.gamma = 1

    def find_max_v_action(self, state_v):
        """
        uses the values and rewards for neighbouring squares, and
        returns action corresponding to the highest value.
        """
        neighbour_v, neighbour_r = self.neighbour_v_r_values(state_v)
        north = neighbour_r[0] + self.gamma * neighbour_v[0]
        east = neighbour_r[1] + self.gamma * neighbour_v[1]
        south = neighbour_r[2] + self.gamma * neighbour_v[2]
        west = neighbour_r[3] + self.gamma * neighbour_v[3]
        max_v = max([north, east, south, west])
        action_index = [north, east, south, west].index(max_v)
        return max_v, action_index

    def neighbour_v_r_values(self, state):
        """
        returns 2 lists for value and rewards for neighbouring squares
        """
        if (state + self.num_cols) < self.num_cells:
            v_n = self.v[state + self.num_cols]
            r_n = self.rewards[state + self.num_cols]
        else:
            v_n = self.v[state]
            r_n = self.rewards[state]
        if (state + 1) % self.num_cols > 0:
            v_e = self.v[state + 1]
            r_e = self.rewards[state + 1]
        else:
            v_e = self.v[state]
            r_e = self.rewards[state]
        if (state - self.num_cols) >= 0:
            v_s = self.v[state - self.num_cols]
            r_s = self.rewards[state - self.num_cols]
        else:
            v_s = self.v[state]
            r_s = self.rewards[state]
        if (state - 1) % self.num_cols < self.num_cols - 1:
            v_w = self.v[state - 1]
            r_w = self.rewards[state - 1]
        else:
            v_w = self.v[state]
            r_w = self.rewards[state]
        return [v_n, v_e, v_s, v_w], [r_n, r_e, r_s, r_w]

    def value_iteration(self):
        """
        applies value iteration algorithm to iterate over all non-terminal states until delta < theta
        """
        delta = 0
        while True:
            delta = 0
            for s in range(0, len(self.v)):
                if s not in self.terminal_states:
                    state_v = self.v[s]
                    max_v, action = self.find_max_v_action(s)
                    self.v[s] = max_v
                    self.policy[s] = self.available_actions[action]
                    delta = max([delta, abs(self.v[s] - state_v)])        
            if delta < self.theta:
                break
                
        return self.v, self.policy

env1 = Deterministic()
v, policy = env1.value_iteration()

for i in range(20, -5, -5):
    print("{} {} {} {} {}".format(v[i], v[i+1], v[i+2], v[i+3], v[i+4]))
    
for i in range(20, -5, -5):
    print("{} {} {} {} {}".format(policy[i], policy[i+1], policy[i+2], policy[i+3], policy[i+4]))

7.0 8.0 9.0 0.0 9.0
6.0 7.0 8.0 0.0 8.0
5.0 6.0 7.0 6.0 7.0
4.0 5.0 6.0 5.0 6.0
3.0 4.0 5.0 4.0 5.0
e e e n w
n n n n n
n n n e n
n n n n n
n n n n n


**Example Test Cell**

In the code cell below, we have provided an example of the type of test code that we will use to mark your work. In these tests, we first check that your `policy` and `v` variables are of the correct type, and then check that their values match the solution. In the future, the test code will be hidden from you.

You must not delete or modify test cells in any way - any modifications you do make will be overwritten at run-time.

In [2]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 1 is tested here.

import numpy as np

# We're giving you the solution values for Exercise 1, but not telling you how to compute them!
solution_values = [3.0, 4.0, 5.0, 4.0, 5.0,
                   4.0, 5.0, 6.0, 5.0, 6.0,
                   5.0, 6.0, 7.0, 6.0, 7.0,
                   6.0, 7.0, 8.0, 0.0, 8.0,
                   7.0, 8.0, 9.0, 0.0, 9.0]
solution_values = np.array(solution_values)

# We're giving you the solution policy for Exercise 1, but not telling you how to compute it!
solution_policy = [
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'e', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'e', 'e', 'e', 'n', 'w',]
solution_policy = np.array(solution_policy)

# Check that policy and v are numpy arrays.
assert(isinstance(policy, np.ndarray))
assert(isinstance(v, np.ndarray))

# Check correct shapes of numpy arrays.
assert(policy.shape == (25, ))
assert(v.shape == (25, ))

# Check whether the numpy arrays have the correct data types.
assert(np.issubdtype(policy.dtype, np.unicode_)) # policy.dtype should be '<U1'
assert(np.issubdtype(v.dtype, np.float64))

# Check whether policy contains only "n", "w", "s", or "e" values.
assert(np.all(np.isin(policy, np.array(["n", "w", "s", "e"]))))

# Print student's solution and true solution for easier comparison / spotting of errors.
print("Student's policy:")
print(np.flip(policy.reshape((5, 5)), 0))
print("Solution policy:")
print(np.flip(solution_policy.reshape((5, 5)), 0))

print("Student's v:")
print(np.flip(v.reshape((5, 5)), 0))
print("Solution v:")
print(np.flip(solution_values.reshape((5, 5)), 0))

# Compare policy (only on states that have a single optimal direction).
states_to_check =  np.array([4, 9, 14, 17, 19, 20, 22, 24])
np.testing.assert_array_equal(policy[states_to_check], solution_policy[states_to_check])

# Compare state_values (also for terminal states --- they have to be zero!).
states_to_check = np.delete(np.arange(25), np.array([18, 23]))
np.testing.assert_array_almost_equal(v[states_to_check], solution_values[states_to_check], decimal=3)

Student's policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Solution policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Student's v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]
Solution v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]


## Exercise 2: Stochastic Environment (12 Marks)

In this exercise, we introduce stochasticity into the environment. Now, the agent is not always able to execute its actions as intended.

With probability 0.8, the agent moves as intended. However, with probability 0.2, it moves in a random direction.

For example, from grid square 0, if the agent tries to move north, with probability 0.8 the action will work as intended. But with probability 0.2, the agent's motor control system will move it in a random direction (including north). So, it will randomly try to move west, east, north or south with probability 0.05 each. Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [3]:
# Please write your code for Exercise 2 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.
import numpy as np

class Stochastic:
    def __init__(self):
        # gridworld dimensions
        self.num_rows = 5
        self.num_cols = 5
        self.num_cells = self.num_cols * self.num_rows

        # item positions
        self.bomb_positions = np.array([18])
        self.gold_positions = np.array([23])
        self.terminal_states = np.array([self.bomb_positions, self.gold_positions])
        
        # reward is -1 for moving into grid square with an additional reward of +/- 10 for the terminal states
        self.rewards = np.array([-1]*self.num_cells)
        self.rewards[self.bomb_positions] += -10
        self.rewards[self.gold_positions] += 10

        self.available_actions = ["n", "e", "s", "w"]

        # state values are initially zero and policy has been initialised with optimal policy "n"
        self.v = np.zeros(self.num_cells)
        self.policy = np.array(["n"]*self.num_cells)
        self.theta = 1e-10
        self.gamma = 1
        # probability of agent not moving as intended is 0.2
        self.unintended = 0.2
        self.intended = 1 - 0.2

    def find_max_v_action(self, state_v):
        """
        uses the values and rewards for neighbouring squares, and returns action corresponding to 
        the highest value. In stochastic environment, value of each direction is 0.8 probability 
        intended action and 0.2 random action (with 0.25 chance of each of the 4 actions
        """
        neighbour_v, neighbour_r = self.neighbour_v_r_values(state_v)
        north = neighbour_r[0] + self.gamma * neighbour_v[0]
        east = neighbour_r[1] + self.gamma * neighbour_v[1]
        south = neighbour_r[2] + self.gamma * neighbour_v[2]
        west = neighbour_r[3] + self.gamma * neighbour_v[3]
        # for stochastic environment, 0.8 prob of intended action and 0.2 prob of random action
        st_north = self.intended*north+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        st_east = self.intended*east+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        st_south = self.intended*south+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        st_west = self.intended*west+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        max_v = max([st_north, st_east, st_south, st_west])
        action_index = [st_north, st_east, st_south, st_west].index(max_v)
        return max_v, action_index

    def neighbour_v_r_values(self, state):
        """
        returns 2 lists for value and rewards for neighbouring squares
        """
        if (state + self.num_cols) < self.num_cells:
            v_n = self.v[state + self.num_cols]
            r_n = self.rewards[state + self.num_cols]
        else:
            v_n = self.v[state]
            r_n = self.rewards[state]
        if (state + 1) % self.num_cols > 0:
            v_e = self.v[state + 1]
            r_e = self.rewards[state + 1]
        else:
            v_e = self.v[state]
            r_e = self.rewards[state]
        if (state - self.num_cols) >= 0:
            v_s = self.v[state - self.num_cols]
            r_s = self.rewards[state - self.num_cols]
        else:
            v_s = self.v[state]
            r_s = self.rewards[state]
        if (state - 1) % self.num_cols < self.num_cols - 1:
            v_w = self.v[state - 1]
            r_w = self.rewards[state - 1]
        else:
            v_w = self.v[state]
            r_w = self.rewards[state]
        return [v_n, v_e, v_s, v_w], [r_n, r_e, r_s, r_w]

    def value_iteration(self):
        """
        applies value iteration algorithm to iterate over all non-terminal states until delta < theta
        """
        delta = 0
        while True:
            delta = 0
            for s in range(0, len(self.v)):
                if s not in self.terminal_states:
                    state_v = self.v[s]
                    max_v, action = self.find_max_v_action(s)
                    self.v[s] = max_v
                    self.policy[s] = self.available_actions[action]
                    delta = max([delta, abs(self.v[s] - state_v)])     
            if delta < self.theta:
                break
                
        return self.v, self.policy

env2 = Stochastic()
v, policy = env2.value_iteration()

for i in range(20, -5, -5):
    print("{} {} {} {} {}".format(v[i], v[i+1], v[i+2], v[i+3], v[i+4]))
    
for i in range(20, -5, -5):
    print("{} {} {} {} {}".format(policy[i], policy[i+1], policy[i+2], policy[i+3], policy[i+4]))

6.0416932891286494 7.2875663582680215 8.613599508716593 0.0 8.692623107335235
4.861851113759854 5.990875869781758 6.370824307347232 0.0 6.467215932034224
3.6755093764709437 4.696213883972035 4.994418628980848 3.2189157994407624 5.102509883951255
2.486995335069779 3.409459887700942 3.669229671301549 2.6412293326632748 3.7861011510512865
1.35979207866988 2.197336720132774 2.4287875129945746 1.5727216146527712 2.552024510140129
e e e n w
n n n n n
n n n e n
n n n e n
n n n n n


In [4]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 2 is tested here.


## Exercise 3: Stochastic Environment with Two Pieces of Gold (8 marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this exercise, we have modified the stochastic environment presented in exercise 2. A second piece of gold has been placed on grid square 12. The terminal state is reached only when **all** pieces of gold are collected or when the bomb is activated.

Compute the optimal policy for this altered environment using Value Iteration.

Hint: You will need to change your state representation in order to account for the additional piece of gold.

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in the previous exercises. These arrays should specify the expected return and an optimal policy at the corresponding grid sqaure **before any pieces of gold are collected or a bomb is activated.** 

In [5]:
# Please write your code for Exercise 3 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.
import numpy as np

class Stochastic2Gold:
    def __init__(self):
        # gridworld dimensions
        self.num_rows = 5
        self.num_cols = 5
        self.num_cells = self.num_cols * self.num_rows

        # item positions - 2 gold
        self.bomb_positions = [18]
        self.gold_positions_2 = [12, 23]
        
        # item positions - 1 gold at 23
        self.gold_positions_23 = [23]
        
        # item positions - 1 gold at 12
        self.gold_positions_12 = [12]
        self.terminal_states = [self.bomb_positions, [self.bomb_positions[0], self.gold_positions_23[0]], [self.bomb_positions[0], self.gold_positions_12[0]]]
        
        """
        rewards, values and policy work as a list of 3 nested lists, each of size 25, where the 1st list is
        state with both gold, 2nd is state with 1 gold at 23 and 3rd is state with 1 gold in 12
        """
        
        # reward is -1 for moving into grid square with an additional reward of +/- 10 for the terminal states
        self.rewards = [[-1 for _ in range(0,self.num_cells)] for _ in range(0,3)]
        self.rewards[0][self.bomb_positions[0]] += -10
        self.rewards[1][self.bomb_positions[0]] += -10
        self.rewards[2][self.bomb_positions[0]] += -10
        self.rewards[0][self.gold_positions_2[0]] += 10
        self.rewards[0][self.gold_positions_2[1]] += 10
        self.rewards[1][self.gold_positions_23[0]] += 10
        self.rewards[2][self.gold_positions_12[0]] += 10

        self.available_actions = ["n", "e", "s", "w"]

        # state values are initially zero and policy has been initialised with optimal policy "n"
        self.v = [[0 for _ in range(0,self.num_cells)] for _ in range(0,3)]
        self.policy = [["n" for _ in range(0,self.num_cells)] for _ in range(0,3)]
        self.theta = 1e-10
        self.gamma = 1
        # probability of agent not moving as intended is 0.2
        self.unintended = 0.2
        self.intended = 1 - 0.2

    def find_max_v_action(self, gold_state, square):
        """
        uses the values and rewards for neighbouring squares, and returns action corresponding to 
        the highest value. In stochastic environment, value of each direction is 0.8 probability 
        intended action and 0.2 random action (with 0.25 chance of each of the 4 actions
        """
        neighbour_v, neighbour_r = self.neighbour_v_r_values(gold_state, square)
        north = neighbour_r[0] + self.gamma * neighbour_v[0]
        east = neighbour_r[1] + self.gamma * neighbour_v[1]
        south = neighbour_r[2] + self.gamma * neighbour_v[2]
        west = neighbour_r[3] + self.gamma * neighbour_v[3]
        # for stochastic environment, 0.8 prob of intended action and 0.2 prob of random action
        st_north = self.intended*north+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        st_east = self.intended*east+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        st_south = self.intended*south+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        st_west = self.intended*west+self.unintended*(0.25*north+0.25*east+0.25*south+0.25*west)
        max_v = max([st_north, st_east, st_south, st_west])
        action_index = [st_north, st_east, st_south, st_west].index(max_v)
        return max_v, action_index

    def neighbour_v_r_values(self, gold_state, square):
        """
        returns 2 lists for value and rewards for neighbouring squares. In state with both golds, neighbouring
        squares to 12 and 23 calculate value for corresponding square in state where the gold on that
        square has been collected.
        """
        if gold_state == 0 and (square == 11 or square == 13 or square == 17 or square == 7):
            if square == 11:
                v_n = self.v[gold_state][square + self.num_cols]
                r_n = self.rewards[gold_state][square + self.num_cols]
                # when moving east, value at square 12 from v[1][12]
                # but reward from v[0][12] because reward of +10 must be collected first before transitioning
                v_e = self.v[gold_state + 1][square + 1]
                r_e = self.rewards[gold_state][square + 1]
                v_s = self.v[gold_state][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
                v_w = self.v[gold_state][square - 1]
                r_w = self.rewards[gold_state][square - 1]
            if square == 13:
                v_n = self.v[gold_state][square + self.num_cols]
                r_n = self.rewards[gold_state][square + self.num_cols]
                v_e = self.v[gold_state][square + 1]
                r_e = self.rewards[gold_state][square + 1]
                v_s = self.v[gold_state][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
                v_w = self.v[gold_state + 1][square - 1]
                r_w = self.rewards[gold_state][square - 1]
            if square == 17:
                v_n = self.v[gold_state][square + self.num_cols]
                r_n = self.rewards[gold_state][square + self.num_cols]
                v_e = self.v[gold_state][square + 1]
                r_e = self.rewards[gold_state][square + 1]
                v_s = self.v[gold_state + 1][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
                v_w = self.v[gold_state][square - 1]
                r_w = self.rewards[gold_state][square - 1]
            if square == 7:
                v_n = self.v[gold_state + 1][square + self.num_cols]
                r_n = self.rewards[gold_state][square + self.num_cols]
                v_e = self.v[gold_state][square + 1]
                r_e = self.rewards[gold_state][square + 1]
                v_s = self.v[gold_state][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
                v_w = self.v[gold_state][square - 1]
                r_w = self.rewards[gold_state][square - 1]
        elif gold_state == 0 and (square == 22 or square == 24):
            if square == 22:
                v_n = self.v[gold_state][square]
                r_n = self.rewards[gold_state][square]
                v_e = self.v[gold_state + 2][square + 1]
                r_e = self.rewards[gold_state][square + 1]
                v_s = self.v[gold_state][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
                v_w = self.v[gold_state][square - 1]
                r_w = self.rewards[gold_state][square - 1]
            if square == 24:
                v_n = self.v[gold_state][square]
                r_n = self.rewards[gold_state][square]
                v_e = self.v[gold_state][square]
                r_e = self.rewards[gold_state][square]
                v_s = self.v[gold_state][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
                v_w = self.v[gold_state + 2][square - 1]
                r_w = self.rewards[gold_state][square - 1]
        else:
            if (square + self.num_cols) < self.num_cells:
                v_n = self.v[gold_state][square + self.num_cols]
                r_n = self.rewards[gold_state][square + self.num_cols]
            else:
                v_n = self.v[gold_state][square]
                # with both golds, square 23 not terminal
                # if hitting wall in 23, reward is -1 as gold is not collected without transitioning here
                if gold_state == 0 and square == 23:
                    r_n = -1
                else:
                    r_n = self.rewards[gold_state][square]
            if (square + 1) % self.num_cols > 0:
                v_e = self.v[gold_state][square + 1]
                r_e = self.rewards[gold_state][square + 1]
            else:
                v_e = self.v[gold_state][square]
                r_e = self.rewards[gold_state][square]
            if (square - self.num_cols) >= 0:
                v_s = self.v[gold_state][square - self.num_cols]
                r_s = self.rewards[gold_state][square - self.num_cols]
            else:
                v_s = self.v[gold_state][square]
                r_s = self.rewards[gold_state][square]
            if (square - 1) % self.num_cols < self.num_cols - 1:
                v_w = self.v[gold_state][square - 1]
                r_w = self.rewards[gold_state][square - 1]
            else:
                v_w = self.v[gold_state][square]
                r_w = self.rewards[gold_state][square]
        return [v_n, v_e, v_s, v_w], [r_n, r_e, r_s, r_w]

    def value_iteration(self):
        """
        applies value iteration algorithm to iterate over all non-terminal states until delta < theta.
        Each iteration loops over all 3 states until convergence.
        """
        delta = 0
        while True:
            delta = 0
            for b in range(0, len(self.v)):
                for s in range(0, self.num_cells):
                    if s not in self.terminal_states[b]:
                        state_v = self.v[b][s]
                        max_v, action = self.find_max_v_action(b, s)
                        self.v[b][s] = max_v
                        self.policy[b][s] = self.available_actions[action]
                        delta = max([delta, abs(self.v[b][s] - state_v)])       
            if delta < self.theta:
                break
                
        return np.array(self.v[0]), np.array(self.policy[0])

env3 = Stochastic2Gold()
v, policy = env3.value_iteration()

for i in range(20, -5, -5):
    print("{} {} {} {} {}".format(v[i], v[i+1], v[i+2], v[i+3], v[i+4]))
print("\n")
for i in range(20, -5, -5):
    print("{} {} {} {} {}".format(policy[i], policy[i+1], policy[i+2], policy[i+3], policy[i+4]))
print("\n")

10.65103993864426 11.796034329270997 13.008487564535121 10.74288293365086 12.970487142269317
11.186135297996916 12.329323720408519 12.512146398880807 0.0 10.615685562832166
12.287027476377382 13.593656375413076 12.480726592729345 12.41930415713496 11.199744275232904
11.175228371177273 12.35165961880423 13.590922916549713 12.281222174794424 11.051284995299435
10.064098056432398 11.174882706353376 12.280459844725018 11.108164762112207 9.993893663954541


e e e w w
e s s n n
e e w w w
e n n w w
n n n n w




In [6]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 3 is tested here.
